In [1]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt

import statsmodels.api as sm
from tqdm import tqdm

from multi_run_v3.data_pp import DataPreprocess
from multi_run_v3.momentum_strategy_group import *

2023-08-26 14:38:29,248	INFO worker.py:1625 -- Started a local Ray instance.


In [2]:
data = pd.read_pickle("Data/cmktcap_data_final.pickle")
days_lst = ["MON","TUE","WED","THU","FRI","SAT","SUN"]

data_pp = DataPreprocess()
data_pp.initialize(data=data)
data_pp.make_mask(mktcap_thresh=1_000_000,
                  vol_thresh=1_000_000,
                  ma=True)

In [3]:
outer_dict = {}
for day in days_lst:
    group_rtn_dict = make_group_mask_rtn(price_df=data_pp.price_df,
                                         weekly_rtn_df=data_pp.weekly_rtn_df,
                                         mask_df=data_pp.mask,
                                         n_group=5,
                                         day_of_week=day,
                                         reb='1',
                                         coin_group=20,
                                         look_back=14)
    inner_dict = {}
    for q in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']: #  'Q6', 'Q7', 'Q8', 'Q9', 'Q10']
        # 해당 리턴으로 Q5가 되었다 -> 전주의 mktcap으로 weight 줘야함
        q_rtn = group_rtn_dict[q] # 이 리턴으로 Q5가 됨. 

        # 이번주 Q5에 속한 코인들의 이번주 리턴 (지난주 mktcap으로 weight)
        # 12월 3일의 mktcap으로 계산한 weight와 12월 10일의 리턴을 곱해준 weighted return을 계산한다
        weekly_mktcap_pp = data_pp.mktcap_df.loc[q_rtn.index]
        prev_mask = q_rtn.shift(-1).isna().replace({False:1,
                                                     True:np.nan})
        prev_weight = (weekly_mktcap_pp * prev_mask).apply(lambda x: x/np.nansum(x), axis=1)
        prev_rtn = (prev_weight.shift(1) * q_rtn).sum(1)

        # 이번주 Q5에 속했던 코인들의 다음주 리턴
        # 이번주 mktcap으로 weight
        next_week_q = q_rtn.shift(1).isna().replace({True:np.nan,
                                                       False:1})
        weekly_rtn_pp = data_pp.weekly_rtn_df.loc[q_rtn.index]
        aft_weight = (weekly_mktcap_pp.shift(1) * next_week_q).apply(lambda x: x / np.nansum(x), axis=1)
        aft_rtn = ((weekly_rtn_pp * next_week_q) * aft_weight).sum(1)

        cc = pd.concat([prev_rtn,aft_rtn.shift(-1)],
                       axis=1,
                       keys=['prev', 'aft']).iloc[1:-1]

        #ols = sm.OLS(cc.aft, sm.add_constant(cc.prev))
        #result = ols.fit()
        #print(q)
        #print(result.summary())
        inner_dict[q] = cc
        
    outer_dict[day] = inner_dict

In [4]:
for q in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']: # 'Q6', 'Q7', 'Q8', 'Q9', 'Q10']
    c = pd.DataFrame()
    for day in days_lst:
        cc = outer_dict[day][q] 
        c = pd.concat([c,cc])
        
    ols = sm.OLS(c.aft, sm.add_constant(c.prev))
    result = ols.fit()
    print(q)
    print(result.summary2())

Q1
                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     -0.000    
Dependent Variable: aft              AIC:                -2009.6486
Date:               2023-08-26 14:41 BIC:                -1998.4975
No. Observations:   1950             Log-Likelihood:     1006.8    
Df Model:           1                F-statistic:        0.02648   
Df Residuals:       1948             Prob (F-statistic): 0.871     
R-squared:          0.000            Scale:              0.020869  
---------------------------------------------------------------------
             Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
---------------------------------------------------------------------
const       -0.0107     0.0053   -2.0036   0.0453   -0.0212   -0.0002
prev        -0.0034     0.0208   -0.1627   0.8707   -0.0441    0.0374
-------------------------------------------------------------------
Omnibus:             383.409       Durbin-Watson:    